In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
# import seaborn as sns
# sns.set()
# sns.set_style("white")
# sns.set_palette("GnBu_d")

# also import these "new" libraries 
# Note: you may have to download an add them to your environment (using e.g. 'conda install -c conda-forge folium')

import folium
from folium import plugins
from folium.plugins import HeatMap
from datetime import datetime #for working with times objects
from datetime import timedelta #for working with times objects
import math
import random
from pathlib import Path

import geopy
from geopy.geocoders import GoogleV3
from geopy.distance import geodesic

In [2]:
df2 = pd.read_csv("../all_station_id.csv")

In [3]:
df = pd.read_csv("../boston_2016_distance_duration_speed.csv")

In [4]:
df.drop(["Unnamed: 0"], axis=1, inplace=True)
# Drop Trips with the 8D OPS stations
df.drop([598580], axis=0, inplace=True)
df.drop([598729], axis=0, inplace=True)
df.drop([1072609], axis=0, inplace=True)
#########################################

# Löschen alle die schneller als 30km/h sind
df.drop([88647], axis=0, inplace=True)
df.drop([88649], axis=0, inplace=True)
df.drop([17997], axis=0, inplace=True)
df.drop([89416], axis=0, inplace=True)
df.drop([824312], axis=0, inplace=True)
df.drop([1143845], axis=0, inplace=True)
#########################################

# Löschen alle die speed < 0 haben
df.drop([1114233], axis=0, inplace=True)
df.drop([1114235], axis=0, inplace=True)
df.drop([1114234], axis=0, inplace=True)
#########################################

# Löscht alle Roundtrips - machen wa doch nicht 
#df = df.drop(df[df['Distance (KM)'] == 0].index)
# Löscht alle Trips die länger als 8Std sind
df = df.drop(df[df['Duration (Seconds)'] > 28800].index)
# Löscht alle Trips die kürzer als 1min sind - gibt keine die kürzer als 61sek sind :)
#df = df.drop(df[df['Duration (Seconds)'] <= 60].index)

In [5]:
# dingo = (df.where(lambda x: (x.start_station_id == 8.0), axis = 0))
# len(df) - dingo['start_station_id'].isnull().sum()

In [6]:
DasDingo = []

In [7]:
for i in df2.start_station_id:
    dingo = (df.where(lambda x: (x.start_station_id == i), axis = 0))
    DasDingo.append(len(df) - dingo['start_station_id'].isnull().sum())

In [8]:
print(DasDingo)

[78, 4682, 8984, 7078, 11683, 4583, 4161, 9527, 9633, 8583, 8006, 7782, 8781, 2618, 15059, 6572, 7232, 10937, 12085, 31457, 14556, 7714, 8444, 7963, 5012, 3726, 6305, 5392, 9246, 13884, 9458, 16666, 1323, 9057, 9235, 10849, 8637, 13991, 11082, 7783, 10113, 13220, 13982, 8053, 7675, 8577, 2838, 9234, 17112, 11682, 12002, 1489, 779, 14333, 7965, 15834, 12788, 3335, 9640, 2995, 5077, 35610, 24085, 6247, 8798, 5790, 10651, 11774, 23132, 11462, 14270, 2190, 5537, 6478, 21157, 10468, 2928, 9132, 5358, 10353, 13154, 7870, 12048, 14647, 1062, 1532, 5514, 11446, 10042, 9823, 11378, 3528, 10449, 2276, 3213, 6404, 12277, 512, 14403, 8596, 12788, 9476, 1707, 1370, 2449, 3230, 10212, 9703, 7147, 5525, 4015, 8807, 4849, 3602, 2449, 2651, 1052, 1044, 2479, 722, 3708, 2836, 5973, 3418, 4382, 1613, 935, 6157, 2904, 6630, 3138, 6288, 1887, 3027, 6254, 5156, 7585, 6504, 0, 0, 1739, 4677, 5041, 394, 4949, 1078, 2865, 431, 2406, 504, 1051, 1990, 3313, 11739, 15218, 17475, 2402, 404, 782, 565, 4693, 17984, 

In [9]:
#df['amout_of_trips_per_station'] = DasDingo(lambda x: (x.DasDingo), axis = 0)
df2['amout_of_trips_per_station'] = pd.DataFrame (DasDingo, columns = ['amout_of_trips_per_station'])

In [10]:
df2

,Unnamed: 0,start_station_id,start_station_name,start_station_latitude,start_station_longitude,amout_of_trips_per_station
0,4748,1,18 Dorrance Warehouse,42.387151,-71.075978,78
1,26681,3,Colleges of the Fenway,42.340021,-71.100812,4682
2,97049,4,Tremont St. at Berkeley St.,42.345392,-71.069616,8984
3,161457,5,Northeastern U / North Parking Lot,42.341814,-71.090179,7078
4,103358,6,Cambridge St. at Joy St.,42.361174,-71.065142,11683
...,...,...,...,...,...,...
184,29748,215,The Eddy at New Street,42.370785,-71.044262,45
185,72713,216,Glendon St at Condor St,42.382447,-71.030232,43
186,71084,217,Orient Heights T Stop - Bennington St at Sarat...,42.386732,-71.006212,70
187,30227,218,Watermark Seaport - Boston Wharf Rd at Seaport...,42.351586,-71.045693,1413


In [11]:
sum(df2['amout_of_trips_per_station'])

1233600

In [12]:
df2.sort_values(['amout_of_trips_per_station'],axis=0, ascending=False,inplace=True,na_position='first')

In [13]:
df2.head()

,Unnamed: 0,start_station_id,start_station_name,start_station_latitude,start_station_longitude,amout_of_trips_per_station
61,97694,67,MIT at Mass Ave / Amherst St,42.358100,-71.093198,35610
19,57721,22,South Station - 700 Atlantic Ave.,42.352175,-71.055547,31457
62,41216,68,Central Square at Mass Ave / Essex St,42.365070,-71.103100,24085
68,20008,74,Harvard Square at Mass Ave/ Dunster,42.373268,-71.118579,23132
74,6903,80,MIT Stata Center at Vassar St / Main St,42.361962,-71.092053,21157


In [14]:
df2["start_coordinates"] = list(zip(df2["start_station_latitude"],df2["start_station_longitude"]))

In [15]:
center_point = (42.36041042227672,-71.05799197093351)
type(center_point)

tuple

In [16]:
def calc_distances_from_center(start_coordinates):
    return geopy.distance.geodesic( start_coordinates,center_point).km

In [17]:
df2['distance_center_(km)'] = df2.apply(lambda row: calc_distances_from_center(row.start_coordinates), axis=1)

In [24]:
# df2.sort_values(['amout_of_trips_per_station'],axis=0, ascending=False,inplace=True,na_position='first')
df2.sort_index(inplace=True)
df2.head(5)

,Unnamed: 0,start_station_id,start_station_name,start_station_latitude,start_station_longitude,amout_of_trips_per_station,start_coordinates,distance_center_(km)
0,4748,1,18 Dorrance Warehouse,42.387151,-71.075978,78,"(42.387151, -71.075978)",3.319275
1,26681,3,Colleges of the Fenway,42.340021,-71.100812,4682,"(42.340021, -71.100812)",4.192547
2,97049,4,Tremont St. at Berkeley St.,42.345392,-71.069616,8984,"(42.345392, -71.069616)",1.923611
3,161457,5,Northeastern U / North Parking Lot,42.341814,-71.090179,7078,"(42.341814, -71.090179)",3.361583
4,103358,6,Cambridge St. at Joy St.,42.361174,-71.065142,11683,"(42.361174, -71.065142)",0.595103


In [25]:
# filepath = Path('/Users/daniel/Desktop/boston_2016_distance_duration_speed.csv')
filepath = Path('C:/Users/rfrei/Desktop/all_station_id_amounts_distance.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
df2.to_csv(filepath)